In [2]:
import json
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

data = []
for filename in glob('../results/2D*.json'):
    with open(filename, 'r') as f:
        data += json.load(f)
print("There are",len(data),"entries.")

There are 91260 entries.


In [ ]:
data=[entry for entry in data if not entry['error']]
print("There are",len(data),"entries with no errors.")

In [ ]:
data[0]

In [ ]:
for entry in data:
    validTimes=np.array(sorted(entry['times'])[1:-1])
    entry['mean_time']=np.mean(validTimes)
    entry['stddev']=np.std(validTimes)
    entry['gflops']=(((entry['N']**3)/3)/entry['mean_time'])/(10**9)
    
data[0]

In [ ]:
sixThreadData=[entry for entry in data if entry['num_threads']==6 and not entry['tiled']]
def groupByKey(dataArray, key):
    permutations={entry[key] for entry in dataArray}
    permutations={perm: [] for perm in permutations}
    for entry in dataArray:
        permutations[entry[key]].append(entry)
    return permutations
permutations=groupByKey(sixThreadData, 'permutation')
print(permutations['kij'][0]['command']==permutations['kij'][2]['command'])

In [ ]:
def plotByKeys(data, keyX, keyY, label=None):
    xs=[entry[keyX] for entry in data]
    ys=[entry[keyY] for entry in data]
    plt.scatter(xs, ys, label=label, alpha=.3)

for permutation in permutations:
    plotByKeys(permutations[permutation], 'N', 'gflops', permutation)

plt.legend()
plt.title("Performance by problem size for different permutations (six threads, no tiles)")
plt.xlabel('Matrix side length')
plt.ylabel('Performance (Gflops/s)')
plt.show()

It's pretty obvious that kij is the best bet among the six-threaded things. Let's make sure that six threads has the best performance.

In [ ]:
threadGroupedData=groupByKey([entry for entry in data if not entry['tiled']], 'num_threads')
for threadNum in threadGroupedData:
    plotByKeys(threadGroupedData[threadNum], 'N', 'gflops', str(threadNum))
    
plt.legend()
plt.title("Performance by number of threads for different numbers of threads (no tiles)")
plt.xlabel('Matrix side length')
plt.ylabel('Performance (Gflops/s)')
plt.show()

Surprisingly close to the fives on occasion, but yes utilizing the most threads gives us the most peformance

Let's compare one thread (with openmp) to one thread (without openmp)

In [ ]:
singleThreadData={'openmp (one thread)': threadGroupedData[1], 'sequential':threadGroupedData[None]}
for threadNum in singleThreadData:
    plotByKeys(singleThreadData[threadNum], 'N', 'gflops', str(threadNum))
    
plt.legend()
plt.title("Performance by problem size and use of openmp (no tiles)")
plt.xlabel('Matrix side length')
plt.ylabel('Performance (Gflops/s)')
plt.show()

This is a pretty shocking result- how does openmp provide negative overhead, and why is it so much better than sequential?

Regardless, let's look into tile sizes on kij, which was our best sequential result

In [ ]:
kijData=[entry for entry in data if entry['permutation']=='kij' and entry['num_threads']==6]
for entry in kijData:
    if entry['TS']==None:
        entry['TS']=entry['N']
        
    
plotByKeys(kijData, 'TS', 'gflops')

plt.title("Performance by tile size for kij (six threads)")
plt.xlabel('Tile size')
plt.ylabel('Performance (Gflops/s)')
plt.show()

So we want to be looking in the 100-200 range. 175ish appears to best for some things, 100ish for others. Let's look for the max and use that as our 'best'.

In [ ]:
bestKIJ=max(kijData, key=lambda entry: entry['gflops'])
print(bestKIJ)
bestKIJTS=bestKIJ['TS']
print(bestKIJTS)

Interestingly, it looks like our best bet on this problem size is to not tile at all- maybe we're looking for tiles that are way too small? Also potentially our small tile sizes have issues where they're multiples of 16 and therefore using 16 times as much space. 176\*16 >2500, so the 2500 matrix could theoretically be just using cache space more efficiently.

### Baseline

In [6]:
ours = []
for filename in glob('../results/2D*.json'):
    with open(filename, 'r') as f:
        ours += json.load(f)


def get_subset(data, permutation, tiled, parallel, threads):
    if not parallel and threads:
        return None
    return [d for d in data if
            d['permutation'] == permutation and d['tiled'] == tiled and d['parallel'] == parallel and d[
                'num_threads'] == threads]

def create_point(x):
    perm_map = {'ijk':1, 'ikj':2, 'jik':3, 'jki':4, 'kij':5, 'kji':6}
    # N, permutation, tiled, TS, parallel, loop_parallelized, num_threads, time
    N = x['N']
    permutation = perm_map[x['permutation']]
    tiled = 1 if x['tiled'] else 0
    TS = x['TS']
    parallel = 1 if x['parallel'] else 0
    loop_parallelized = 0 if x['loop_parallelized']=='I' else 0
    num_threads = x['num_threads']
    time = x['time']
    return np.array([N, permutation, tiled, TS, parallel, loop_parallelized, num_threads]), time

OURS = {}
for permutation in ['ijk', 'ikj', 'jik', 'jki', 'kij', 'kji']:
    for tiled in [True, False]:
        for parallel in [True, False]:
            for threads in [None, 1, 2, 3, 4, 5, 6]:
                our_data = get_subset(ours, permutation, tiled, parallel, threads)
                if not our_data:
                    continue
                for m in our_data:
                    m['time'] = np.mean(sorted(m['times'])[1:-1])
                
                P, y = create_point(our_data[0])
                Y = np.array([y])
                for x in our_data[1:]:
                    p, y = create_point(x)
                    P = np.vstack([P, p])
                    Y = np.vstack([Y, y])

                print(P)
                '''
                for N in data:
                    times = [(x['time'], 'num_threads={}'.format(x['num_threads'])) for x in data[N]]
                    times.sort()
                    min_time = times[0]
                    time = min_time[0]
                    num_threads = min_time[1]
                    D.append((N, time, num_threads))
                    if N == 1000 and tiled and parallel and threads==6:
                        print((permutation, tiled, parallel, threads), '->', times)
                        
                D.sort()
                X = [i[0] for i in D]
                Y = [i[1] for i in D]
                # compute effective GFLOPs
                G = []
                for point in D:
                    N, time, _ = point
                    G.append((N ** 3) / 3 / time / (10 ** 9))
                OURS[(permutation, tiled, parallel, threads)] = {'X': X, 'Y': Y, 'G': G, 'D': D}
                '''


[[1024    1    1 ...    1    0    1]
 [1024    1    1 ...    1    0    1]
 [1024    1    1 ...    1    0    1]
 ...
 [ 512    1    1 ...    1    0    1]
 [ 512    1    1 ...    1    0    1]
 [ 512    1    1 ...    1    0    1]]
[[1024    1    1 ...    1    0    2]
 [1024    1    1 ...    1    0    2]
 [1024    1    1 ...    1    0    2]
 ...
 [ 512    1    1 ...    1    0    2]
 [ 512    1    1 ...    1    0    2]
 [ 512    1    1 ...    1    0    2]]
[[1024    1    1 ...    1    0    3]
 [1024    1    1 ...    1    0    3]
 [1024    1    1 ...    1    0    3]
 ...
 [ 512    1    1 ...    1    0    3]
 [ 512    1    1 ...    1    0    3]
 [ 512    1    1 ...    1    0    3]]
[[1024    1    1 ...    1    0    4]
 [1024    1    1 ...    1    0    4]
 [1024    1    1 ...    1    0    4]
 ...
 [ 512    1    1 ...    1    0    4]
 [ 512    1    1 ...    1    0    4]
 [ 512    1    1 ...    1    0    4]]
[[1024    1    1 ...    1    0    5]
 [1024    1    1 ...    1    0    5]
 [1024    1   

[[1024 2 1 ... 0 0 None]
 [1024 2 1 ... 0 0 None]
 [1024 2 1 ... 0 0 None]
 ...
 [512 2 1 ... 0 0 None]
 [512 2 1 ... 0 0 None]
 [512 2 1 ... 0 0 None]]
[[1024 2 0 None 1 0 1]
 [1024 2 0 None 1 0 1]
 [2560 2 0 None 1 0 1]
 [2560 2 0 None 1 0 1]
 [2500 2 0 None 1 0 1]
 [2500 2 0 None 1 0 1]
 [5000 2 0 None 1 0 1]
 [5000 2 0 None 1 0 1]
 [3584 2 0 None 1 0 1]
 [3584 2 0 None 1 0 1]
 [3072 2 0 None 1 0 1]
 [3072 2 0 None 1 0 1]
 [1536 2 0 None 1 0 1]
 [1536 2 0 None 1 0 1]
 [500 2 0 None 1 0 1]
 [500 2 0 None 1 0 1]
 [2048 2 0 None 1 0 1]
 [2048 2 0 None 1 0 1]
 [4000 2 0 None 1 0 1]
 [4000 2 0 None 1 0 1]
 [1500 2 0 None 1 0 1]
 [1500 2 0 None 1 0 1]
 [4500 2 0 None 1 0 1]
 [4500 2 0 None 1 0 1]
 [1000 2 0 None 1 0 1]
 [1000 2 0 None 1 0 1]
 [4608 2 0 None 1 0 1]
 [4608 2 0 None 1 0 1]
 [4096 2 0 None 1 0 1]
 [4096 2 0 None 1 0 1]
 [2000 2 0 None 1 0 1]
 [2000 2 0 None 1 0 1]
 [3000 2 0 None 1 0 1]
 [3000 2 0 None 1 0 1]
 [512 2 0 None 1 0 1]
 [512 2 0 None 1 0 1]]
[[1024 2 0 None 1 0 2]

[[1024    4    1 ...    1    0    1]
 [1024    4    1 ...    1    0    1]
 [1024    4    1 ...    1    0    1]
 ...
 [ 512    4    1 ...    1    0    1]
 [ 512    4    1 ...    1    0    1]
 [ 512    4    1 ...    1    0    1]]
[[1024    4    1 ...    1    0    2]
 [1024    4    1 ...    1    0    2]
 [1024    4    1 ...    1    0    2]
 ...
 [ 512    4    1 ...    1    0    2]
 [ 512    4    1 ...    1    0    2]
 [ 512    4    1 ...    1    0    2]]
[[1024    4    1 ...    1    0    3]
 [1024    4    1 ...    1    0    3]
 [1024    4    1 ...    1    0    3]
 ...
 [ 512    4    1 ...    1    0    3]
 [ 512    4    1 ...    1    0    3]
 [ 512    4    1 ...    1    0    3]]
[[1024    4    1 ...    1    0    4]
 [1024    4    1 ...    1    0    4]
 [1024    4    1 ...    1    0    4]
 ...
 [ 512    4    1 ...    1    0    4]
 [ 512    4    1 ...    1    0    4]
 [ 512    4    1 ...    1    0    4]]
[[1024    4    1 ...    1    0    5]
 [1024    4    1 ...    1    0    5]
 [1024    4   

[[1024 5 1 ... 0 0 None]
 [1024 5 1 ... 0 0 None]
 [1024 5 1 ... 0 0 None]
 ...
 [512 5 1 ... 0 0 None]
 [512 5 1 ... 0 0 None]
 [512 5 1 ... 0 0 None]]
[[1024 5 0 None 1 0 1]
 [1024 5 0 None 1 0 1]
 [2560 5 0 None 1 0 1]
 [2560 5 0 None 1 0 1]
 [2500 5 0 None 1 0 1]
 [2500 5 0 None 1 0 1]
 [5000 5 0 None 1 0 1]
 [5000 5 0 None 1 0 1]
 [3584 5 0 None 1 0 1]
 [3584 5 0 None 1 0 1]
 [3072 5 0 None 1 0 1]
 [3072 5 0 None 1 0 1]
 [1536 5 0 None 1 0 1]
 [1536 5 0 None 1 0 1]
 [500 5 0 None 1 0 1]
 [500 5 0 None 1 0 1]
 [2048 5 0 None 1 0 1]
 [2048 5 0 None 1 0 1]
 [4000 5 0 None 1 0 1]
 [4000 5 0 None 1 0 1]
 [1500 5 0 None 1 0 1]
 [1500 5 0 None 1 0 1]
 [4500 5 0 None 1 0 1]
 [4500 5 0 None 1 0 1]
 [1000 5 0 None 1 0 1]
 [1000 5 0 None 1 0 1]
 [4608 5 0 None 1 0 1]
 [4608 5 0 None 1 0 1]
 [4096 5 0 None 1 0 1]
 [4096 5 0 None 1 0 1]
 [2000 5 0 None 1 0 1]
 [2000 5 0 None 1 0 1]
 [3000 5 0 None 1 0 1]
 [3000 5 0 None 1 0 1]
 [512 5 0 None 1 0 1]
 [512 5 0 None 1 0 1]]
[[1024 5 0 None 1 0 2]

In [ ]:
mkl_data = []
for filename in glob('../results/baseline.mkl.json'):
    with open(filename, 'r') as f:
        mkl_data += json.load(f)
for m in mkl_data:
    m['type'] = m['command'].split('/')[-1].split(' ')[0]
    m['time'] = np.mean(m['times'][1:-1])

Ns = set()
for m in mkl_data:
    if m['N'] not in Ns:
        Ns.add(m['N'])


data = {}
for type in ['TT_MKL', 'TS_MKL', 'SS_MKL']:
    mkl_data_type = [m for m in mkl_data if m['type']==type]
    data[type] = {}
    for N in Ns:
        data[type][N] = []
        for m in mkl_data_type:
            if m['N'] == N:
                data[type][N].append(m)

D = {'TT_MKL': [], 'TS_MKL': [], 'SS_MKL': []}
X = {'TT_MKL': [], 'TS_MKL': [], 'SS_MKL': []}
Y = {'TT_MKL': [], 'TS_MKL': [], 'SS_MKL': []}
Labels = {'TT_MKL': [], 'TS_MKL': [], 'SS_MKL': []}
for type in data:
    for N in data[type]:
        times = [(x['time'], 'num_threads={}'.format(x['num_threads'])) for x in data[type][N]]
        times.sort()
        min_time = times[0]
        time = min_time[0]
        num_threads = min_time[1]
        D[type].append((N, time, num_threads))
    D[type].sort()
    X[type] = [i[0] for i in D[type]]
    Y[type] = [i[1] for i in D[type]]
    Labels[type] = [i[2] for i in D[type]]

# compute effective GFLOPs
G = {'TT_MKL': [], 'TS_MKL': [], 'SS_MKL': []}
for point in D['SS_MKL']:
    N, time, _ = point
    G['SS_MKL'].append(2*(N**3) / time / (10**9))
for type in ['TS_MKL', 'TT_MKL']:
    for point in D[type]:    
        G[type].append((N**3) / time / (10**9))

plt.figure(figsize=(20,8))
plt.rcParams.update({'font.size': 14})
plt.subplot(1, 2, 1)
for type, c in zip(['TT_MKL', 'SS_MKL'],['go-', 'bo-', 'ro-']):
    plt.plot(X[type], Y[type], c, linewidth=2, label=type)
plt.title('MKL data')
plt.xlabel('Problem size (N)')
plt.ylabel('Time (sec)')
plt.legend()

plt.subplot(1, 2, 2)
for type, c in zip(['TT_MKL', 'SS_MKL'],['go-', 'bo-', 'ro-']):
    plt.plot(X[type], G[type], c, linewidth=2, label=type)

plt.title('MKL data')
plt.xlabel('Problem size (N)')
plt.ylabel('Effective GFLOPs')
plt.legend()

plt.show()

In [ ]:
# mkl gflops
plt.figure(figsize=(20,8))
plt.rcParams.update({'font.size': 14})

for type, c in zip(['TT_MKL', 'SS_MKL'],['go-', 'bo-', 'ro-']):
    plt.plot(X[type], Y[type], c, linewidth=2, label=type)
    
# overlay our data
#for permutation in ['ijk', 'ikj', 'jik', 'jki', 'kij', 'kji']:
for permutation in ['ikj', 'ijk']:
    print(permutation)
    tiled = True
    parallel = True
    JAX = OURS[(permutation, tiled, parallel, 6)]
    if permutation == 'ijk':
        plt.plot(JAX['X'], JAX['Y'], 'o-', label=permutation)
    else:
        ys = [y+1 for y in JAX['Y']]
        plt.plot(JAX['X'], ys, 'o-', label=permutation)

plt.title('Effective GFLOPs vs. Problem Size')
plt.xlabel('Problem size (N)')
plt.ylabel('Time (sec)')
#plt.ylabel('Effective GFLOPs')
plt.legend()

plt.show()